In [1]:
import pandas as pd
import lightningchart as lc
import numpy as np
import trimesh
import time
from math import radians
from threading import Thread
from scipy.signal import spectrogram

lc.set_license('my-license-key')

dataset_info = {
    '../Dataset/0D.csv': '0D',
    '../Dataset/1D.csv': '1D',
    '../Dataset/2D.csv': '2D',
    '../Dataset/3D.csv': '3D',
    '../Dataset/4D.csv': '4D',
}

vibration_columns = ['Vibration_1', 'Vibration_2', 'Vibration_3']

summary_data = {}
dataframes = []
for path, identifier in dataset_info.items():
    df = pd.read_csv(path)
    df['Dataset_ID'] = identifier
    dataframes.append(df)
    summary_data[identifier] = [df[col].mean() for col in vibration_columns]

all_data = pd.concat(dataframes, ignore_index=True)

filtered_data = all_data[all_data['Measured_RPM'] > 0]
sampled_data = filtered_data.iloc[::100].reset_index(drop=True)
data_4D = pd.read_csv('../Dataset/4D.csv')
data_full = data_4D[data_4D['Measured_RPM'] >= 0].reset_index(drop=True)
data_normalized = data_4D[data_4D['Measured_RPM'] >= 0].iloc[::30].reset_index(drop=True)
data_normalized_100 = data_4D[data_4D['Measured_RPM'] >= 0].iloc[::100].reset_index(drop=True)

vibration_columns = ['Vibration_1', 'Vibration_2', 'Vibration_3']

In [2]:
max_values = {col: max([summary_data[dataset][i] for dataset in summary_data]) for i, col in enumerate(vibration_columns)}
normalized_summary_data = {
    name: [value / max_values[vibration_columns[i]] for i, value in enumerate(values)]
    for name, values in summary_data.items()
}

# Dashboard of Line & Barchart & Spider Chart

In [8]:
average_vibrations = sampled_data.groupby('Dataset_ID')[['Vibration_1', 'Vibration_2', 'Vibration_3']].mean()

average_vibrations['Average_Vibration'] = average_vibrations.mean(axis=1)

unbalance_info = {
    '0D': {'Radius': 0, 'Mass': 0},
    '1D': {'Radius': 14, 'Mass': 3.281},
    '2D': {'Radius': 18.5, 'Mass': 3.281},
    '3D': {'Radius': 23, 'Mass': 3.281},
    '4D': {'Radius': 23, 'Mass': 6.614},
}

unbalance_labels = [
    f"{key} (R={info['Radius']}, M={info['Mass']})" for key, info in unbalance_info.items()
]
average_values = average_vibrations['Average_Vibration'].tolist()
bar_data = [{'category': label, 'value': value} for label, value in zip(unbalance_labels, average_values)]

dashboard = lc.Dashboard(rows=2, columns=2, theme=lc.Themes.Light)

chart_avg_vibration = dashboard.BarChart(
    row_index=0,  # Place in the first row
    column_index=0,  # Spans the single column
    vertical=True,
)
chart_avg_vibration.set_title('Average Vibration Intensity by Unbalance Strength')
chart_avg_vibration.set_title_font(16,weight='bold')
chart_avg_vibration.set_data(bar_data)

# Group data for the line chart
grouped_data = {}
for key, info in unbalance_info.items():
    group = sampled_data[sampled_data['Dataset_ID'] == key].copy() 
    grouped_data[key] = group

chart_spider = dashboard.SpiderChart(
    row_index=0,  # Place in the first row
    column_index=1,  # Spans the single column
)

chart_spider.set_axis_label_font(weight='bold', size=15)
chart_spider.set_title('Comparative Analysis of Unbalance Strengths').set_title_font(16,weight='bold')
chart_spider.set_nib_style(thickness=5, color=lc.Color(0, 0, 0))
chart_spider.set_web_mode("circle")
chart_spider.set_web_count(8)
                    
for metric in vibration_columns:
    chart_spider.add_axis(metric)

series_list = []
for dataset_name, values in normalized_summary_data.items():
    series = chart_spider.add_series()
    series.set_name(dataset_name)
    series.add_points([{'axis': metric, 'value': value} for metric, value in zip(vibration_columns, values)])
    series_list.append(series)

legend = chart_spider.add_legend().set_dragging_mode("draggable")
for series in series_list:
    legend.add(data=series)

chart_rpm_vibration = dashboard.ChartXY(
    row_index=1,  # Place in the second row
    column_index=0,  # Spans the single column
    column_span=2,
    title='Impact of Unbalance on Vibration (Sensor 1)'
)
chart_rpm_vibration.set_title_font(16,weight='bold')

legend = chart_rpm_vibration.add_legend().set_dragging_mode("draggable")
for key, group in grouped_data.items():
    unbalance = unbalance_info[key]
    series = chart_rpm_vibration.add_line_series()
    series.set_name(f"{key} (R={unbalance['Radius']} mm, M={unbalance['Mass']} g)")
    series.add(group['Measured_RPM'].tolist(), group['Vibration_1'].tolist())
    legend.add(series)

chart_rpm_vibration.get_default_x_axis().set_title('RPM')
chart_rpm_vibration.get_default_y_axis().set_title('Vibration Intensity')

dashboard.open(method="browser")

127.0.0.1 - - [07/Jan/2025 11:41:34] "GET / HTTP/1.1" 200 -


# Line Chart & Dual Y-axis

In [12]:
vibration_columns = ['Vibration_1', 'Vibration_2', 'Vibration_3']

# Initialize chart
chart = lc.ChartXY(
    title="Time Series Analysis of Vibration and RPM for Largest Unbalance",
    theme=lc.Themes.Light,
)
chart.set_title_font(16, weight='bold')
x_axis = chart.get_default_x_axis()
x_axis.set_title("Time (Index)")
x_axis.set_interval(0, len(data_normalized_100), stop_axis_after=True)

# Remove default Y-axis
chart.get_default_y_axis().dispose()

# Add custom Y-axes and line series
legend = chart.add_legend()
line_series = []
sensors = ['Vibration_1', 'Vibration_2', 'Vibration_3', 'Measured_RPM']

for i, sensor in enumerate(sensors):
    y_axis = chart.add_y_axis(stack_index=i)  # Create a separate Y-axis for each sensor
    y_axis.set_title(sensor)
    if sensor in vibration_columns:  # Adjust Y-axis range for log-scaled values
        y_axis.set_interval(-0.15, 0.15, stop_axis_after=True)        
        
    series = chart.add_line_series(y_axis=y_axis, data_pattern='ProgressiveX')
    series.set_name(sensor)
    legend.add(series)
    line_series.append(series)

# Add data to the corresponding line series
for i, series in enumerate(line_series):
    sensor_column = sensors[i]
    series.add(
        x=list(range(len(data_normalized_100))),
        y=data_normalized_100[sensor_column].tolist()
    )

# Open chart
chart.open(method="browser")

127.0.0.1 - - [07/Jan/2025 11:44:18] "GET / HTTP/1.1" 200 -


# Histogram

In [32]:
dashboard = lc.Dashboard(rows=1, columns=3, theme=lc.Themes.Light)

# Add histograms for each vibration column with logarithmic binning
for i, col in enumerate(vibration_columns):
    # Prepare histogram data with logarithmic binning
    values = data_full[col].values
    min_val = max(np.min(values), 1e-10)  # Avoid zero/negative values for log bins
    max_val = np.max(values)
    log_bins = np.logspace(np.log10(min_val), np.log10(max_val), num=30)
    counts, bin_edges = np.histogram(values, bins=log_bins)
    
    # Remove bins with zero values
    bar_data = [
        {"category": f"{bin_edges[j]:.2e}–{bin_edges[j+1]:.2e}", "value": int(count)}
        for j, count in enumerate(counts) if count > 0  # Filter out zero counts
    ]

    # Create a BarChart for the histogram
    chart = dashboard.BarChart(
        row_index=0,
        column_index=i,
        vertical=True,
    )
    chart.set_title(f'Logarithmic Binned Distribution of Vibration Values of {col}').set_title_font(16,weight='bold')
    chart.set_data(bar_data)
    chart.set_sorting('disabled')  # Ensure bars appear in order
    chart.set_palette_colors(
        steps=[
            {'value': 0, 'color': lc.Color('blue')},  # Low value
            {'value': 0.5, 'color': lc.Color('yellow')},  # Mid value
            {'value': 1, 'color': lc.Color('red')}  # High value
        ],
        percentage_values=True  # Normalize values to percentages (0 to 1 range)
    )
dashboard.open(method="browser")


127.0.0.1 - - [07/Jan/2025 12:32:49] "GET / HTTP/1.1" 200 -


# Line series of Sensor 1 Vibration over Time

In [29]:

sample_index = range(len(data_normalized_100))  # Generate sample indices
vibration_amplitude = data_normalized_100['Vibration_1']  # Extract Vibration_1 data

chart = lc.ChartXY(
    title="Dataset 4D - Largest Unbalance",
    theme=lc.Themes.Light
)
chart.set_title_font(16,weight='bold')
# Add a line series to the chart
line_series = chart.add_line_series().add(sample_index, vibration_amplitude)
line_series.set_name("Vibration Sensor 1 Amplitude").set_line_color(lc.Color(0, 0, 255)).set_palette_line_coloring(
    steps=[
        {'value': -0.05, 'color': lc.Color('blue')},  # Low value
        {'value': 0, 'color': lc.Color('yellow')},  # Mid value
        {'value': 0.05, 'color': lc.Color('red')}  # High value
    ],
    look_up_property='y',
    percentage_values=True
)

# Customize axes labels
chart.get_default_x_axis().set_title("Sample Index")
chart.get_default_y_axis().set_title("Vibration Sensor 1 Amplitude")

chart.get_default_y_axis().set_interval(-0.1, 0.1, stop_axis_after=True)

chart.open(method="browser")


127.0.0.1 - - [07/Jan/2025 12:23:35] "GET / HTTP/1.1" 200 -


# Heatmap for FFT of Dataset 4D 

In [30]:
# Extract relevant columns
vibration = data_normalized['Vibration_1'].values
rotation_speed = data_normalized['Measured_RPM'].values

# Ensure vibration data is non-zero and has valid values
if len(vibration) == 0 or np.all(vibration == 0):
    print("Vibration data is invalid or contains only zeros.")
else:
    # Sampling rate (assumed 4096 Hz)
    fs = 4096

    # Compute spectrogram
    frequencies, times, Sxx = spectrogram(vibration, fs=fs, nperseg=1024, noverlap=512)

    # Normalize the spectrogram
    Sxx = 10 * np.log10(Sxx + 1e-8)  # Convert power to dB

    # Interpolate rotation speed for the time axis
    interp_rpm = np.interp(times, np.linspace(0, len(vibration) / fs, len(rotation_speed)), rotation_speed)

    # Correct potential scaling issues by debugging
    print(f"frequencies range: {frequencies.min()} to {frequencies.max()}")
    print(f"interp_rpm range: {interp_rpm.min()} to {interp_rpm.max()}")
    print(f"Sxx range: {Sxx.min()} to {Sxx.max()}")

    # Create LightningChart heatmap
    chart = lc.ChartXY(
        title="FFT of Dataset 4D (Vibration_1), Second Cycle - Largest Unbalance",
        theme=lc.Themes.Light
    )
    chart.set_title_font(16,weight='bold')
    # Add heatmap series
    heatmap = chart.add_heatmap_grid_series(
        rows=len(frequencies),
        columns=len(times),
    )

    # Set heatmap bounds (matching the frequency and interpolated RPM ranges)
    heatmap.set_start(x=frequencies.min(), y=interp_rpm.min())
    heatmap.set_end(x=frequencies.max(), y=interp_rpm.max())

    # Set heatmap intensity values (transpose to align correctly)
    heatmap.invalidate_intensity_values(Sxx.T.tolist())

    # Define a custom color palette (matching Matplotlib's "Blues")
    custom_palette = [
        {"value": np.nanmin(Sxx), "color": lc.Color('blue')},
        {"value": np.nanpercentile(Sxx, 25), "color": lc.Color('cyan')},
        {"value": np.nanmedian(Sxx), "color": lc.Color('green')},
        {"value": np.nanpercentile(Sxx, 75), "color": lc.Color('yellow')},
        {"value": np.nanmax(Sxx), "color": lc.Color('red')},
    ]

    heatmap.set_palette_coloring(
        steps=custom_palette,
        look_up_property='value',
        interpolate=True
    )

    # Customize axes
    chart.get_default_x_axis().set_title("Frequency [Hz]").set_interval(0, 2000)
    chart.get_default_y_axis().set_title("Rotation Speed [RPM]").set_interval(1200, 2000)

    chart.add_legend(data=heatmap).set_title("Amplitude (dB)")

    chart.open(method="browser")


frequencies range: 0.0 to 2048.0
interp_rpm range: 615.37992 to 3830.5617
Sxx range: -79.99999990235399 to -11.335400093002887


127.0.0.1 - - [07/Jan/2025 12:24:38] "GET / HTTP/1.1" 200 -


# Dashboard of Mesh Model & Line series & Gauge Chart

In [35]:

vibration_columns = ['Vibration_1', 'Vibration_2', 'Vibration_3']
rpm_column = 'Measured_RPM'

dashboard = lc.Dashboard(
    rows=3,
    columns=4,
    theme=lc.Themes.Light
)

mesh_chart = dashboard.Chart3D(
    row_index=0,
    column_index=0,
    row_span=3,
    column_span=2,
    )

mesh_chart.set_title("Rotating Shaft with Vibration Sensors").set_title_font(20, weight='bold')
x_axis_mesh = mesh_chart.get_default_x_axis().set_interval(-1, 1, stop_axis_after=True).set_tick_strategy("Empty")
y_axis_mesh = mesh_chart.get_default_y_axis().set_interval(-1, 1, stop_axis_after=True).set_tick_strategy("Empty")
z_axis_mesh = mesh_chart.get_default_z_axis().set_interval(-1, 1, stop_axis_after=True).set_tick_strategy("Empty")

# Add the motor model
motor_model = mesh_chart.add_mesh_model().set_color(lc.Color('gray'))
motor_obj_path = '../Dataset/scaled motor dc skala.obj'
motor_scene = trimesh.load(motor_obj_path)
if isinstance(motor_scene, trimesh.Scene):
    motor_mesh = motor_scene.dump(concatenate=True)
else:
    motor_mesh = motor_scene

motor_vertices = motor_mesh.vertices.flatten().tolist()
motor_indices = motor_mesh.faces.flatten().tolist()
motor_normals = motor_mesh.vertex_normals.flatten().tolist()
motor_model.set_model_geometry(vertices=motor_vertices, indices=motor_indices, normals=motor_normals)
motor_model.set_scale(0.4).set_model_location(0.1, -0.2, 0).set_model_rotation(0, 180, 0)

# Add the bearing model
bearing_model = mesh_chart.add_mesh_model().set_color(lc.Color('gray'))
bearing_obj_path = '../Dataset/scaled Pillow Block Bearing.stl'
bearing_scene = trimesh.load(bearing_obj_path)
if isinstance(bearing_scene, trimesh.Scene):
    bearing_mesh = bearing_scene.dump(concatenate=True)
else:
    bearing_mesh = bearing_scene

bearing_vertices = bearing_mesh.vertices.flatten().tolist()
bearing_indices = bearing_mesh.faces.flatten().tolist()
bearing_normals = bearing_mesh.vertex_normals.flatten().tolist()
bearing_model.set_model_geometry(vertices=bearing_vertices, indices=bearing_indices, normals=bearing_normals)
bearing_model.set_scale(0.18).set_model_location(0.6, -0.3, 0).set_model_rotation(0, 90, 0)

# Add the wheel model
wheel_model = mesh_chart.add_mesh_model().set_color(lc.Color('white'))
wheel_obj_path = '../Dataset/scaled reelybuggywheel.stl'
wheel_scene = trimesh.load(wheel_obj_path)
if isinstance(wheel_scene, trimesh.Scene):
    wheel_mesh = wheel_scene.dump(concatenate=True)
else:
    wheel_mesh = wheel_scene

wheel_vertices = wheel_mesh.vertices.flatten().tolist()
wheel_indices = wheel_mesh.faces.flatten().tolist()
wheel_normals = wheel_mesh.vertex_normals.flatten().tolist()
wheel_model.set_model_geometry(vertices=wheel_vertices, indices=wheel_indices, normals=wheel_normals)
wheel_model.set_scale(0.12).set_model_location(0.86, -0.28, 0).set_model_rotation(0, 90, 0)

# Add the sensor models
sensor_obj_path = '../Dataset/Sensor.stl'
sensor_scene = trimesh.load(sensor_obj_path)
if isinstance(sensor_scene, trimesh.Scene):
    sensor_mesh = sensor_scene.dump(concatenate=True)
else:
    sensor_mesh = sensor_scene

sensor_positions = [
    (0.6, -0.32, -0.24),  # Sensor 1
    (0.72, -0.16, 0),      # Sensor 2
    (0.1, 0.29, 0)         # Sensor 3
]

sensor_models = []
for position in sensor_positions:
    sensor_model = mesh_chart.add_mesh_model()
    sensor_vertices = sensor_mesh.vertices.flatten().tolist()
    sensor_indices = sensor_mesh.faces.flatten().tolist()
    sensor_normals = sensor_mesh.vertex_normals.flatten().tolist()
    sensor_model.set_model_geometry(vertices=sensor_vertices, indices=sensor_indices, normals=sensor_normals)
    sensor_model.set_scale(0.001).set_model_location(*position).set_model_rotation(90, 0, 0)
    sensor_models.append(sensor_model)

scale_factor = 0.2
for sensor_model in sensor_models:
    sensor_model.set_palette_coloring(
        steps=[
            {'value': 0 , 'color': lc.Color('lightblue')},
            {'value': 0.25 , 'color': lc.Color('darkblue')},
            {'value': 0.5 , 'color': lc.Color('lightgreen')},
            {'value': 0.75 , 'color': lc.Color('darkgreen')},
            {'value': 1, 'color': lc.Color('yellow')},
            {'value': 2 , 'color': lc.Color('FFB266')},
            {'value': 3, 'color': lc.Color('CC6600')},
            {'value': 4, 'color': lc.Color('red')},
        ],
        look_up_property='value',
        interpolate=True,
    )

wheel_model.set_palette_coloring(
    steps=[
        {'value': 20, 'color': lc.Color('blue')},     # Low RPM: Blue
        {'value': 30, 'color': lc.Color('green')},   # Moderate RPM: Green
        {'value': 40, 'color': lc.Color('yellow')},  # High RPM: Yellow
        {'value': 700, 'color': lc.Color('orange')},  # Very High RPM: Orange
        {'value': 1000, 'color': lc.Color('red')},    # Critical RPM: Red
    ],
    look_up_property='value',
    interpolate=True,)

line_chart = dashboard.ChartXY(
    row_index=0,
    column_index=2,
    row_span=1,
    column_span=2,
    title='Relation between Vibration and RPM'
)
line_chart.set_title_font(20, weight='bold')
# Configure axes
x_axis_line = line_chart.get_default_x_axis().set_title('RPM').set_interval(0, 2000, stop_axis_after=True)
y_axis_line = line_chart.get_default_y_axis().set_title('Vibration Intensity')

legend = line_chart.add_legend()

# Add a series for each vibration column
for vibration_col in vibration_columns:
    series = line_chart.add_line_series()
    series.set_name(vibration_col)
    series.add(data_normalized_100[rpm_column].tolist(), data_normalized_100[vibration_col].tolist())
    legend.add(series)

constant_line = x_axis_line.add_constant_line()

# Create gauge chart for first sensor
vibration_chart_1 = dashboard.GaugeChart(
    row_index=1,
    row_span=1,
    column_index=2,
    column_span=1,
)
vibration_chart_1.set_angle_interval(start=225, end=-45)
vibration_chart_1.set_interval(start=0, end=8)
vibration_chart_1.set_value_indicators([
    {'start': 0, 'end': 2, 'color': lc.Color('green')},
    {'start': 2, 'end': 4, 'color': lc.Color('yellow')},
    {'start': 4, 'end': 6, 'color': lc.Color('orange')},
    {'start': 6, 'end': 8, 'color': lc.Color('red')}
])
vibration_chart_1.set_bar_thickness(25)
vibration_chart_1.set_value_indicator_thickness(12).set_needle_thickness(4).set_tick_font(15, weight='bold')
vibration_chart_1.set_title('Sensor 1 Vibration').set_title_font(15, weight='bold')
vibration_chart_1.set_value_label_font(20, weight='bold')

# Create gauge chart for second sensor
vibration_chart_2 = dashboard.GaugeChart(
    row_index=1,
    row_span=1,
    column_index=3,
    column_span=1,
)
vibration_chart_2.set_angle_interval(start=225, end=-45)
vibration_chart_2.set_interval(start=0, end=8)
vibration_chart_2.set_value_indicators([
    {'start': 0, 'end': 2, 'color': lc.Color('green')},
    {'start': 2, 'end': 4, 'color': lc.Color('yellow')},
    {'start': 4, 'end': 6, 'color': lc.Color('orange')},
    {'start': 6, 'end': 8, 'color': lc.Color('red')}
])
vibration_chart_2.set_bar_thickness(25)
vibration_chart_2.set_value_indicator_thickness(12).set_needle_thickness(4).set_tick_font(15, weight='bold')
vibration_chart_2.set_title('Sensor 2 Vibration').set_title_font(15, weight='bold')
vibration_chart_2.set_value_label_font(20, weight='bold')

# Create gauge chart for third sensor
vibration_chart_3 = dashboard.GaugeChart(
    row_index=2,
    row_span=1,
    column_index=2,
    column_span=1,
)
vibration_chart_3.set_angle_interval(start=225, end=-45)
vibration_chart_3.set_interval(start=0, end=8)
vibration_chart_3.set_value_indicators([
    {'start': 0, 'end': 2, 'color': lc.Color('green')},
    {'start': 2, 'end': 4, 'color': lc.Color('yellow')},
    {'start': 4, 'end': 6, 'color': lc.Color('orange')},
    {'start': 6, 'end': 8, 'color': lc.Color('red')}
])
vibration_chart_3.set_bar_thickness(25)
vibration_chart_3.set_value_indicator_thickness(12).set_needle_thickness(4).set_tick_font(15, weight='bold')
vibration_chart_3.set_title('Sensor 3 Vibration').set_title_font(15, weight='bold')
vibration_chart_3.set_value_label_font(20, weight='bold')

# Create gauge chart for wheel
wheel_chart = dashboard.GaugeChart(
    row_index=2,
    row_span=1,
    column_index=3,
    column_span=1,
)
wheel_chart.set_angle_interval(start=225, end=-45)
wheel_chart.set_interval(start=0, end=1000)
wheel_chart.set_value_indicators([
    {'start': 0, 'end': 250, 'color': lc.Color('green')},
    {'start': 250, 'end': 500, 'color': lc.Color('yellow')},
    {'start': 500, 'end': 750, 'color': lc.Color('orange')},
    {'start': 750, 'end': 1000, 'color': lc.Color('red')}
])
wheel_chart.set_bar_thickness(25)
wheel_chart.set_value_indicator_thickness(12).set_needle_thickness(4).set_tick_font(15, weight='bold')
wheel_chart.set_title('Shaft Rotation (RPM)').set_title_font(15, weight='bold')
wheel_chart.set_value_label_font(20, weight='bold')

animation_delay = 0.02

def update_models():
    accumulated_angle = 0  # To track the cumulative rotation angle
    animation_steps = 20

    for idx, row in data_full.iloc[4090::100].iterrows():
        scaled_vibration_values = [row[vibration_columns[i]] * scale_factor for i in range(len(sensor_models))]
        rpm_value = row[rpm_column]

        for step in range(animation_steps):
            progress = step / (animation_steps - 1)
            direction = -1 if step >= animation_steps // 2 else 1

            # Update sensors
            for i, sensor_model in enumerate(sensor_models):
                base_position = sensor_positions[i]
                offset = direction * scaled_vibration_values[i] * progress * 0.01
                updated_position = (
                    base_position[0],
                    base_position[1] + offset,
                    base_position[2]
                )
                sensor_model.set_model_location(*updated_position)
                sensor_model.set_vertex_values(lambda: [scaled_vibration_values[i]] * len(sensor_mesh.vertices))

            # Update wheel rotation
            angle_increment = (rpm_value / 60) * 360 * animation_delay
            accumulated_angle += angle_increment
            wheel_model.set_model_rotation(accumulated_angle, 90, 0)
            wheel_model.set_vertex_values(lambda: [rpm_value] * len(wheel_mesh.vertices))
            # print(f"Wheel rotated around X-axis with RPM: {rpm_value}, Angle: {accumulated_angle:.2f}")

            # Update gauge charts
            vibration_chart_1.set_value(scaled_vibration_values[0])  # First sensor's vibration
            vibration_chart_2.set_value(scaled_vibration_values[1])  # Second sensor's vibration
            vibration_chart_3.set_value(scaled_vibration_values[2])  # Third sensor's vibration
            wheel_chart.set_value(rpm_value)
            
            constant_line.set_value(rpm_value).set_stroke(2.5, lc.Color('yellow'))

            time.sleep(animation_delay)

        # Reset sensors after animation
        for i, sensor_model in enumerate(sensor_models):
            base_position = sensor_positions[i]
            sensor_model.set_model_location(*base_position)
            sensor_model.set_vertex_values(lambda: [scaled_vibration_values[i]] * len(sensor_mesh.vertices))

dashboard.open(method="browser", live=True)
update_models()

KeyboardInterrupt: 

# Mesh Model & Line Series & Stress Visualization

In [2]:
# Initialize LightningChart Dashboard
dashboard = lc.Dashboard(columns=2, rows=1, theme=lc.Themes.Dark)

# Column 1: 3D Chart
chart3d = dashboard.Chart3D(row_index=0, column_index=0).set_title("Dynamic Force and Vibration Visualization")

# Load motor and bearing block meshes
motor_mesh = chart3d.add_mesh_model()
cylinder_mesh = chart3d.add_mesh_model()
bearing_block_mesh = chart3d.add_mesh_model()
wheel_mesh = chart3d.add_mesh_model()

# Load OBJ/STL files
motor_scene = trimesh.load('../Dataset/scaled Motor without shaft.obj')
cylinder_scene = trimesh.load('../Dataset/scaled shaft.obj')
bearing_block_scene = trimesh.load('../Dataset/scaled Pillow Block Bearing.stl')
wheel_scene = trimesh.load('../Dataset/scaled reelybuggywheel.stl')

# Column 2: Line Chart
chart_line = dashboard.ChartXY(row_index=0, column_index=1).set_title("Vibration and Stress Levels")

chart_line.get_default_y_axis().dispose()  # Remove the default Y-axis
legend = chart_line.add_legend()

# Sensors and parameters to display in the line chart
sensors = ['Sensor 1', 'Sensor 2', 'Sensor 3', 'Bearing Block Stress', 'Cylinder Stress', 'RPM']
line_series = []

# Add line series for each parameter
for i, sensor in enumerate(sensors):
    y_axis = chart_line.add_y_axis(stack_index=i)
    y_axis.set_title(sensor)
    series = chart_line.add_line_series(y_axis=y_axis, data_pattern='ProgressiveX')
    series.set_name(sensor)
    line_series.append(series)
    legend.add(series)

# Function to process meshes
def process_mesh(scene):
    if isinstance(scene, trimesh.Scene):
        mesh = scene.to_geometry()
    else:
        mesh = scene
    vertices = mesh.vertices.flatten().tolist()
    indices = mesh.faces.flatten().tolist()
    normals = mesh.vertex_normals.flatten().tolist()
    return vertices, indices, normals

motor_vertices, motor_indices, motor_normals = process_mesh(motor_scene)
cylinder_vertices, cylinder_indices, cylinder_normals = process_mesh(cylinder_scene)
bearing_block_vertices, bearing_block_indices, bearing_block_normals = process_mesh(bearing_block_scene)
wheel_vertices, wheel_indices, wheel_normals = process_mesh(wheel_scene)

# Set geometry for meshes
motor_mesh.set_model_geometry(vertices=motor_vertices, indices=motor_indices, normals=motor_normals)
motor_mesh.set_scale(0.36).set_model_location(-0.28, -0.17, 0).set_model_rotation(0, 180, 0)

bearing_block_mesh.set_model_geometry(vertices=bearing_block_vertices, indices=bearing_block_indices, normals=bearing_block_normals)
bearing_block_mesh.set_scale(0.16).set_model_location(0.62, -0.3, 0).set_model_rotation(0, 90, 0)

cylinder_mesh.set_model_geometry(vertices=cylinder_vertices, indices=cylinder_indices, normals=cylinder_normals)
cylinder_mesh.set_scale(0.25).set_model_location(0.33, -0.27, 0).set_model_rotation(0, 180, 0)

wheel_mesh.set_model_geometry(vertices=wheel_vertices, indices=wheel_indices, normals=wheel_normals)
wheel_mesh.set_scale(0.1).set_model_location(0.82, -0.28, 0).set_model_rotation(0, 90, 0)

# Add dynamic color palettes
cylinder_mesh.set_palette_coloring(
    steps=[
        {'value': 9, 'color': lc.Color('lightblue')},
        {'value': 9.5, 'color': lc.Color('blue')},    # Low intensity
        {'value': 9.75, 'color': lc.Color('lightgreen')},
        {'value': 10, 'color': lc.Color('green')},  # Moderate intensity
        {'value': 40, 'color': lc.Color('yellow')}, # High intensity
        {'value': 65.8, 'color': lc.Color('orange')}, # Very high intensity
        {'value': 66.8, 'color': lc.Color('red')},
        {'value': 67.5, 'color': lc.Color('red')},    # Critical intensity
    ],
    look_up_property='value',
    interpolate=True
)

bearing_block_mesh.set_palette_coloring(
    steps=[
        {'value': 9, 'color': lc.Color('lightblue')},
        {'value': 9.5, 'color': lc.Color('blue')},    # Low intensity
        {'value': 9.75, 'color': lc.Color('lightgreen')},
        {'value': 10, 'color': lc.Color('green')},  # Moderate intensity
        {'value': 40, 'color': lc.Color('yellow')}, # High intensity
        {'value': 65.8, 'color': lc.Color('orange')}, # Very high intensity
        {'value': 66.8, 'color': lc.Color('red')},
        {'value': 67.5, 'color': lc.Color('red')},    # Critical intensity
    ],
    look_up_property='value',
    interpolate=True
)

wheel_mesh.set_palette_coloring(
    steps=[
        {'value': 20, 'color': lc.Color('blue')},     # Low RPM: Blue
        {'value': 100, 'color': lc.Color('green')},   # Moderate RPM: Green
        {'value': 500, 'color': lc.Color('yellow')},  # High RPM: Yellow
        {'value': 700, 'color': lc.Color('orange')},  # Very High RPM: Orange
        {'value': 1000, 'color': lc.Color('red')},    # Critical RPM: Red
    ],
    look_up_property='value',
    interpolate=True
)

# Function to normalize forces
def normalize_forces(vertex_values, min_value, max_value):
    non_default_values = [v for v in vertex_values if v > min_value]

    if not non_default_values:
        return vertex_values

    min_force = min(non_default_values)
    max_force = max(non_default_values)

    if min_force == max_force:
        return [min_value for _ in vertex_values]

    return [
        (value - min_force) / (max_force - min_force) * (max_value - min_value) + min_value
        if value > min_value else min_value
        for value in vertex_values
    ]

# Stress calculations
def generate_stress_values(vertices, connection_point, max_stress, scale, location):
    stress_values = []
    for i in range(0, len(vertices), 3):
        x = vertices[i] * scale[0] + location[0]
        y = vertices[i + 1] * scale[1] + location[1]
        z = vertices[i + 2] * scale[2] + location[2]
        distance = np.sqrt((x - connection_point[0]) ** 2 + (y - connection_point[1]) ** 2 + (z - connection_point[2]) ** 2)
        stress = max_stress * np.exp(-0.05 * distance)
        stress_values.append(stress)
    return stress_values

# Streaming function
def stream_data():
    accumulated_angle = 0
    cylinder_scale = (0.25, 0.25, 0.25)
    cylinder_location = (0.33, -0.27, 0)
    bearing_block_scale = (0.16, 0.16, 0.16)
    bearing_block_location = (0.62, -0.3, 0)
    wheel_connection = (cylinder_location[0] + cylinder_scale[0], cylinder_location[1], cylinder_location[2])

    time_ms = 0

    for index, row in data_full.iloc[4090::100].iterrows():
        rpm = row['Measured_RPM']
        max_stress = max(10, 2000 / (rpm + 1))

        # Stress values
        cylinder_stress = generate_stress_values(
            cylinder_vertices, wheel_connection, max_stress, cylinder_scale, cylinder_location
        )
        bearing_block_stress = generate_stress_values(
            bearing_block_vertices, wheel_connection, max_stress, bearing_block_scale, bearing_block_location
        )

        # Normalize values
        min_stress = min(cylinder_stress + bearing_block_stress)
        max_stress = max(cylinder_stress + bearing_block_stress)
        cylinder_stress_normalized = normalize_forces(cylinder_stress, min_stress, max_stress)
        bearing_block_stress_normalized = normalize_forces(bearing_block_stress, min_stress, max_stress)

        # Update meshes
        cylinder_mesh.set_vertex_values(lambda: cylinder_stress_normalized)
        bearing_block_mesh.set_vertex_values(lambda: bearing_block_stress_normalized)

        # RPM normalization
        wheel_rpm_normalized = normalize_forces([rpm] * len(wheel_vertices), 20, 1000)
        wheel_mesh.set_vertex_values(lambda: wheel_rpm_normalized)

        # Debugging stress values
        print(f"RPM: {rpm}")
        print(f"Normalized Stress Range: {(min_stress)} to {(max_stress)}")

        # Update line chart with sensor and stress data
        sensor_values = [row['Vibration_1'], row['Vibration_2'], row['Vibration_3']]
        stress_values = [np.mean(bearing_block_stress), np.mean(cylinder_stress), rpm]
        all_values = sensor_values + stress_values

        for i, series in enumerate(line_series):
            series.add([time_ms], [all_values[i]])

        # Update wheel rotation
        angle_increment = (rpm / 60) * 360 * 0.1
        accumulated_angle += angle_increment
        wheel_mesh.set_model_rotation(accumulated_angle, 90, 0)

        time_ms += 1
        time.sleep(0.1)

dashboard.open(live=True, method="browser")

# Start streaming data
thread = Thread(target=stream_data)
thread.daemon = True
thread.start()


RPM: 28.610235
Normalized Stress Range: 65.88049300443217 to 67.54058302960408
RPM: 28.610235
Normalized Stress Range: 65.88049300443217 to 67.54058302960408
RPM: 28.610235
Normalized Stress Range: 65.88049300443217 to 67.54058302960408
RPM: 28.610235
Normalized Stress Range: 65.88049300443217 to 67.54058302960408
RPM: 28.610235
Normalized Stress Range: 65.88049300443217 to 67.54058302960408
RPM: 28.610235
Normalized Stress Range: 65.88049300443217 to 67.54058302960408
RPM: 28.610235
Normalized Stress Range: 65.88049300443217 to 67.54058302960408
RPM: 28.610235
Normalized Stress Range: 65.88049300443217 to 67.54058302960408
RPM: 28.610235
Normalized Stress Range: 65.88049300443217 to 67.54058302960408
RPM: 28.610235
Normalized Stress Range: 65.88049300443217 to 67.54058302960408
RPM: 28.610235
Normalized Stress Range: 65.88049300443217 to 67.54058302960408
RPM: 28.610235
Normalized Stress Range: 65.88049300443217 to 67.54058302960408
RPM: 28.610235
Normalized Stress Range: 65.880493004